In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
# functions used to plot pitch track comparisons
def extract_pitch_track(audio_path, fmin=75, fmax=400):
    """Return pitch track (Hz) and time axis for a given audio file."""
    y, sr = librosa.load(audio_path, sr=None)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr, fmin=fmin, fmax=fmax)
    max_idx = np.argmax(magnitudes, axis=0)
    pitch_track = pitches[max_idx, range(magnitudes.shape[1])]
    times = librosa.times_like(pitch_track, sr=sr)
    return times, pitch_track

def plot_pitch_comparison(gt_path, nmf_path, ica_path, title):
    """Plot three vertically stacked pitch tracks for a given source."""
    fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True, sharey=True)

    for ax, (label, path, color) in zip(
        axes,
        [
            ("Ground Truth", gt_path, "k"),
            ("NMF Output", nmf_path, "b"),
            ("ICA Output", ica_path, "r"),
        ],
    ):
        times, pitch_track = extract_pitch_track(path)
        ax.plot(times, pitch_track, color=color, lw=1)
        ax.set_ylim(0, 400)
        ax.set_ylabel("Pitch (Hz)")
        ax.grid(True)
        ax.set_title(label)

    axes[-1].set_xlabel("Time (s)")
    fig.suptitle(f"Pitch Tracks: {title}", fontsize=14, weight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_pitch_comparison("vocal_gt.wav", "vocal_nmf.wav", "vocal_ica.wav", "Vocal Source")
plot_pitch_comparison("instr_gt.wav", "instr_nmf.wav", "instr_ica.wav", "Instrumental Source")

In [ ]:
#MFCC using python_speech_features (may or may not use)
from python_speech_features import mfcc
import scipy.io.wavfile as wav
fig, ax = plt.subplots()
mfcc_data= np.swapaxes(mfcc_data, 0 ,1)
cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower')
ax.set_title('MFCC')

plt.show()

In [ ]:
#function to plot MFCC
def plot_mfcc_comparison(gt_path, nmf_path, ica_path, title, n_mfcc=13):
    """Plot MFCCs for GT, NMF, ICA results of a given source."""
    fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

    for ax, (label, path) in zip(
        axes,
        [
            ("Ground Truth", gt_path),
            ("NMF Output", nmf_path),
            ("ICA Output", ica_path),
        ],
    ):
        y, sr = librosa.load(path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        librosa.display.specshow(mfccs, x_axis="time", ax=ax, sr=sr)
        ax.set_title(label)
        ax.set_ylabel("MFCC")
    
    axes[-1].set_xlabel("Time (s)")
    fig.suptitle(f"MFCCs: {title}", fontsize=14, weight='bold')
    plt.colorbar(ax=axes, format='%+2.0f dB', shrink=0.6)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_mfcc_comparison("vocal_gt.wav", "vocal_nmf.wav", "vocal_ica.wav", "Vocal Source")
plot_mfcc_comparison("instr_gt.wav", "instr_nmf.wav", "instr_ica.wav", "Instrumental Source")